## One Class SVM Evaluations

In [1]:
from prequential import run_prequential

from river import anomaly
from river.anomaly import OneClassSVM
from river import feature_extraction as fx
from skmultiflow.drift_detection.adwin import ADWIN

from float.feature_selection import FIRES, OFS           # online feature methods
from skmultiflow.data import FileStream             # create stream from file

import pandas as pd

In [2]:
data_loader = FileStream(filepath='../merged_cesnet.csv')

In [3]:
ref_sample, _ = data_loader.next_sample(50)
data_loader.reset()

In [4]:
svm_params = {
    'q': 0.8,
    'gamma': 1,
    'nu': 0.05
}

fires_params = {
    'penalty_s': 0.1,
    'penalty_r': 1,
    'lr_mu': 0.025,
    'lr_sigma': 0.1,
    'n_total_features': data_loader.n_features,
    'n_selected_features': 10,
    'classes': data_loader.target_values,
    'baseline': "gaussian",
    'ref_sample': ref_sample
}

ofs_params = {
    'n_selected_features': 5,
    'n_total_features': data_loader.n_features,
    'baseline': "gaussian",
    'ref_sample': ref_sample    
}

### Without ADWIN

In [5]:
# no feature selection
print("EVALUATING: OC-SVM")
accuracy, precision, recall, f1, auc, avg_processing_time, _ = run_prequential(
    classifier=anomaly.QuantileFilter(
        (fx.RBFSampler(gamma=svm_params['gamma']) | OneClassSVM(nu=svm_params['nu'])),
        q=svm_params['q']
    ), 
    stream=data_loader, 
    feature_selector=None, 
    drift_detection=None, 
    preq_samples=data_loader.n_remaining_samples()
)
print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}, AUC: {auc:.6f}")
print(f"Average processing time: {avg_processing_time}")

# FIRES
print("EVALUATING: OC-SVM + FIRES")
accuracy, precision, recall, f1, auc, avg_processing_time, _, _, _ = run_prequential(
    classifier=anomaly.QuantileFilter(
        (fx.RBFSampler(gamma=svm_params['gamma']) | OneClassSVM(nu=svm_params['nu'])),
        q=svm_params['q']
    ), 
    stream=data_loader, 
    feature_selector=FIRES(**fires_params), 
    drift_detection=None, 
    preq_samples=data_loader.n_remaining_samples()
)
print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}, AUC: {auc:.6f}")
print(f"Average processing time: {avg_processing_time}")

# OFS
print("EVALUATING: OC-SVM + OFS")
accuracy, precision, recall, f1, auc, avg_processing_time, _, _, _ = run_prequential(
    classifier=anomaly.QuantileFilter(
        (fx.RBFSampler(gamma=svm_params['gamma']) | OneClassSVM(nu=svm_params['nu'])),
        q=svm_params['q']
    ),   
    stream=data_loader, 
    feature_selector=OFS(**ofs_params), 
    drift_detection=None, 
    preq_samples=data_loader.n_remaining_samples()
)
print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}, AUC: {auc:.6f}")
print(f"Average processing time: {avg_processing_time}")

EVALUATING: OC-SVM
Accuracy: 0.687178, Precision: 0.621572, Recall: 0.915753, F1: 0.740515, AUC: 0.692784
Average processing time: 0.006403835527450147
EVALUATING: OC-SVM + FIRES


c:\Users\Carlo Alamani\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\Users\Carlo Alamani\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

### With ADWIN

In [ ]:
# no feature selection
print("EVALUATING: OC-SVM + ADWIN")
accuracy, precision, recall, f1, auc, avg_processing_time, drift_idx_list = run_prequential(
    classifier=anomaly.QuantileFilter(
        (fx.RBFSampler(gamma=svm_params['gamma']) | OneClassSVM(nu=svm_params['nu'])),
        q=svm_params['q']
    ),  
    stream=data_loader, 
    feature_selector=None, 
    drift_detection=ADWIN(), 
    preq_samples=data_loader.n_remaining_samples()
)
print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}, AUC: {auc:.6f}")
print(f"Average processing time: {avg_processing_time}")
print(drift_idx_list)

# FIRES
print("EVALUATING: OC-SVM + ADWIN + FIRES")
accuracy, precision, recall, f1, auc, avg_processing_time, _, _, drift_idx_list = run_prequential(
    classifier=anomaly.QuantileFilter(
        (fx.RBFSampler(gamma=svm_params['gamma']) | OneClassSVM(nu=svm_params['nu'])),
        q=svm_params['q']
    ), 
    stream=data_loader, 
    feature_selector=FIRES(**fires_params), 
    drift_detection=ADWIN(), 
    preq_samples=data_loader.n_remaining_samples()
)
print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}, AUC: {auc:.6f}")
print(f"Average processing time: {avg_processing_time}")
print(drift_idx_list)

# OFS
print("EVALUATING: OC-SVM + ADWIN + OFS")
accuracy, precision, recall, f1, auc, avg_processing_time, _, _, drift_idx_list = run_prequential(
    classifier=anomaly.QuantileFilter(
        (fx.RBFSampler(gamma=svm_params['gamma']) | OneClassSVM(nu=svm_params['nu'])),
        q=svm_params['q']
    ), 
    stream=data_loader, 
    feature_selector=OFS(**ofs_params), 
    drift_detection=ADWIN(), 
    preq_samples=data_loader.n_remaining_samples()
)
print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}, AUC: {auc:.6f}")
print(f"Average processing time: {avg_processing_time}")
print(drift_idx_list)

EVALUATING: OC-SVM + ADWIN
